# Implementing recurrent networks

与卷积网络（需要我们实现一种新的操作）不同，理论上来说，循环网络的实现相当直接：尽管像LSTM这样更复杂的循环网络的"cell"的特定细节似乎有点复杂，但它最终只是一组操作符，很容易在自动微分工具中链接在一起。然而，在有效实现循环网络时需要考虑一些因素，大部分源于它们基本上是"顺序"模型的事实。这意味着，与"正常"的深度网络不同，我们认为所有操作在数据批次上是可以很容易并行化的，LSTM的输入数据需要逐个被输入：在处理第一个元素之前我们无法处理第二个元素，依此类推。

## Implementing the LSTM cell

有一点是我们在注释中指出的，我想在这里强调一下，那就是LSTM单元背后的矩阵并没有看起来那么复杂。我们来看一个典型的LSTM单元方程，通常在文档或论文中以这种方式编写，这里以PyTorch文档为例：https://pytorch.org/docs/stable/generated/torch.nn.LSTM.html 

\begin{array}{ll} \\
        i_t = \sigma(W_{ii} x_t + b_{ii} + W_{hi} h_{t-1} + b_{hi}) \\
        f_t = \sigma(W_{if} x_t + b_{if} + W_{hf} h_{t-1} + b_{hf}) \\
        g_t = \tanh(W_{ig} x_t + b_{ig} + W_{hg} h_{t-1} + b_{hg}) \\
        o_t = \sigma(W_{io} x_t + b_{io} + W_{ho} h_{t-1} + b_{ho}) \\
        c_t = f_t \odot c_{t-1} + i_t \odot g_t \\
        h_t = o_t \odot \tanh(c_t) \\
    \end{array}

这些方程看起来相当令人困惑（而且下标中的$W$项有错误）。但首先要意识到，并且我在注释中强调过的是，实际上并没有8个不同的权重，只有两个不同的权重。你应该将向量的视角

\begin{bmatrix}
i \\ f \\ g \\ o
\end{bmatrix}

不是四个单独的向量，而是一个长度是隐藏单元四倍的单一向量。也就是说，我们实际上有更新门（update gate）的权重向量和遗忘门（forget gate）的权重向量，它们只是在单个向量中的不同部分。
\begin{equation}
\begin{bmatrix}
i \\ f \\ g \\ o
\end{bmatrix}
= \begin{pmatrix} \sigma \\ \sigma \\ \tanh \\ \sigma \end{pmatrix} (W_{hi} x + W_{hh} h + b)
\end{equation}
（在这里，我单方面决定摒弃关于使用两个不同偏差项的奇怪选择，并正确重新标记了$W$的下标）。

无论如何，这就是PyTorch在内部的实现方式，如果您检查实际的类，就可以看到这一点。


In [104]:
import torch
import torch.nn as nn
model = nn.LSTMCell(20,100)
print(model.weight_hh.shape)
print(model.weight_ih.shape)

torch.Size([400, 100])
torch.Size([400, 20])


We could define our own LSTM cell using something like the following function.

In [105]:
import numpy as np

def sigmoid(x):
    return 1/(1+np.exp(-x))

def lstm_cell(x, h, c, W_hh, W_ih, b):
    i,f,g,o = np.split(W_ih@x + W_hh@h + b, 4)
    i,f,g,o = sigmoid(i), sigmoid(f), np.tanh(g), sigmoid(o)
    c_out = f*c + i*g
    h_out = o * np.tanh(c_out)
    return h_out, c_out

Let's confirm that this gives the same results as PyTorch's version.

In [106]:
x = np.random.randn(1,20).astype(np.float32)
h0 = np.random.randn(1,100).astype(np.float32)
c0 = np.random.randn(1,100).astype(np.float32)

h_, c_ = model(torch.tensor(x), (torch.tensor(h0), torch.tensor(c0)))

In [107]:
h, c = lstm_cell(x[0], h0[0], c0[0], 
                 model.weight_hh.detach().numpy(), 
                 model.weight_ih.detach().numpy(), 
                 (model.bias_hh + model.bias_ih).detach().numpy())

print(np.linalg.norm(h_.detach().numpy() - h), 
      np.linalg.norm(c_.detach().numpy() - c))

2.7782272e-07 5.9651194e-07


### Iterating over a sequence

We can run the function on a whole sequence simply by iterating over this cell.

In [108]:
model = nn.LSTM(20, 100, num_layers = 1)

In [109]:
X = np.random.randn(50,20).astype(np.float32)
h0 = np.random.randn(1,100).astype(np.float32)
c0 = np.random.randn(1,100).astype(np.float32)
H_, (hn_, cn_) = model(torch.tensor(X)[:,None,:], 
                       (torch.tensor(h0)[:,None,:], 
                        torch.tensor(c0)[:,None,:]))

In [110]:
def lstm(X, h, c, W_hh, W_ih, b):
    H = np.zeros((X.shape[0], h.shape[0]))
    for t in range(X.shape[0]):
        h, c = lstm_cell(X[t], h, c, W_hh, W_ih, b)
        H[t,:] = h
    return H, c
        

In [111]:
H, cn = lstm(X, h0[0], c0[0], 
             model.weight_hh_l0.detach().numpy(), 
             model.weight_ih_l0.detach().numpy(), 
             (model.bias_hh_l0 + model.bias_ih_l0).detach().numpy())

In [111]:
print(np.linalg.norm(H - H_[:,0,:].detach().numpy()),
      np.linalg.norm(cn - cn_[0,0,:].detach().numpy()))

7.889616690880754e-07 2.1517381e-07


## Batching efficiently

在上面的例子中，您可能会注意到一些非常奇怪的张量大小。我们需要将一个大小为`(50,1,20)`的张量传递给PyTorch的LSTM，以获得与我们自己的函数类似的结果（更直观大小的结果）。这是怎么回事呢？

基本问题在于，正如前面提到的，我们在自己的函数中实现的LSTM本质上是顺序的：我们需要在将时间t=1的`lstm_cell`调用运行之前，才能继续到时间t=2、时间t=3等，因为在时间t=2时计算的隐藏单元将作为时间t=2的输入。然而，正如我们之前多次看到的那样，上面执行的每个`lstm_cell`调用操作本质上都是一个矩阵-向量操作，理想情况下，我们希望将其转换为矩阵-矩阵操作，以便进行更高效的计算。

因此，就像多层感知机（MLP）的情况一样，我们将使用小批量来实现这种计算效率。但关键点在于，这些小批量的示例不能来自同一个序列：它们需要来自多个_不同的_序列（或者在实践中通常来自单个序列中相距很远的位置）。

### The form of batched samples for LSTMs

一旦我们转向样本的小批量，每个样本都有一定数量的时间步长（暂时假设每个样本的时间步长相同，但这将很快解决），那么我们就需要存储具有批次、时间和输入维度的输入。最自然的情况是，它看起来像下面这样：

    X[NUM_BATCHES][NUM_TIMESTEPS][INPUT_SIZE]
    
我们可以称之为NTC格式（这对于LSTMs来说并不常见，但类似于我们讨论过的NHWC格式用于图像）。然而，PyTorch在LSTMs中原生地使用"TNC"格式，即按照以下顺序存储张量：

    X[NUM_TIMESTEPS][NUM_BATCHES][INPUT_SIZE]
    
为什么会这样呢？在PyTorch的其他设置中，批次维度总是第一维度，事实上，现在甚至有一个选项可以使用“批次优先”格式来处理LSTM，尽管它不是默认设置。

由于内存局部性的原因。为了将LSTM的操作有效地批处理为矩阵乘积形式，我们希望执行矩阵乘法。


\begin{equation}
\begin{bmatrix}
I \\ F \\ G \\ O
\end{bmatrix}
= \begin{pmatrix} \sigma \\ \sigma \\ \tanh \\ \sigma \end{pmatrix} (X W_{hi} + H W_{hh} + b)
\end{equation}



这里我们将$X$视为一个$N \times C$矩阵，$H, I, F, G, O$视为一个$N \times K$矩阵，其中$K$是隐藏维度（我们隐式地对矩阵进行了转置，这就是为什么PyTorch在文档中将它们列为$W_{ih}$的原因，即使对于它显示的向量形式来说，这并不完全准确）

为了使每个$X$和$H$（在所有批次和单个时间步中）在内存中是连续的，我们需要使用THC的顺序，然后选择`X[t]`和`H[t]`作为相关索引。让我们看看在实现中它是如何呈现的。

In [112]:
def lstm_cell(x, h, c, W_hh, W_ih, b):
    i,f,g,o = np.split(x@W_ih + h@W_hh + b[None,:], 4, axis=1)
    i,f,g,o = sigmoid(i), sigmoid(f), np.tanh(g), sigmoid(o)
    c_out = f*c + i*g
    h_out = o * np.tanh(c_out)
    return h_out, c_out

def lstm(X, h, c, W_hh, W_ih, b):
    H = np.zeros((X.shape[0], X.shape[1], h.shape[1]))
    for t in range(X.shape[0]):
        h, c = lstm_cell(X[t], h, c, W_hh, W_ih, b)
        H[t,:,:] = h
    return H, c

In [113]:
X = np.random.randn(50,80,20).astype(np.float32)
h0 = np.random.randn(80,100).astype(np.float32)
c0 = np.random.randn(80,100).astype(np.float32)

In [114]:
H_, (hn_, cn_) = model(torch.tensor(X), 
                       (torch.tensor(h0)[None,:,:], 
                        torch.tensor(c0)[None,:,:]))

In [115]:
H, cn = lstm(X, h0, c0,
             model.weight_hh_l0.detach().numpy().T, 
             model.weight_ih_l0.detach().numpy().T, 
             (model.bias_hh_l0 + model.bias_ih_l0).detach().numpy())

In [116]:
print(np.linalg.norm(H - H_.detach().numpy()),
      np.linalg.norm(cn - cn_[0].detach().numpy()))

7.64154016618813e-06 1.8674904e-06


If we _were_ to store the matrices in NTC ordering, and e.g., using the matrix multiplication we will consider in  Homework 3, where matrices need to be compact in memory before performing multiplication, we would have to be copying memory around during each update.  The TNC format fixes this (and even if we _were_ to develop a more efficient multiplication strategy that could directly consider strided matrices, the NTC format would still sacrifice memory locality).  PyTorch (and needle, as you will implement on Homework 4) will thus the TNC route, and sacrifice a few people being confused the first time they use LSTMs.

### Packed sequences

There is still one substantial problem with the approach above: the tensor form requires that each sequence in the batch be the same size.  This is often explicitly not the case for RNNs, where e.g., one may want to use an RNN to process individual sentences of text, individual audio signals, etc.  A large _benefit_ of the RNN for these settings is that the sequences can all be different lengths, yet an RNN can process them all similarly.  Thus, we have a common setting where the different sequences in a minibatch might be different lengths.

One "simple" way to deal with this is simply to zero-pad the input sequences to the size of the longest sequence.  That is, we can place all the sequences in a single `X[MAX_TIMESTEPS][BATCH][DIMENION]`, replace all inputs that occur after the end of each sequence with zeros, and then after-the-fact extract the hidden unit representation at the effective end of each sequence (or at all valid points in the sequence).  Since this takes advantage of "full, equal sized" matrix multiplications at each step, this is reasonable solution for it's simplicity).

However, if the sequences in a batch are _very_ different sizes, it should be acknowledged that this can ultimately be inefficient to run all the operations of the LSTM on what amount to a lot of meaningless data full of just zeros.  To get around this, an alternative is to support "packed sequences".  This represents the input as a 2D tensor
    
    X[sum_batch TIMESTEPS(batch)][DIMENSION]

that lumps together elements in both the batch and time dimensions.

In order to still exploit contiguous memory, we still want to group together elements by timestep, so this format contains first all inputs for all sequences at time 1 (they will all exist here across all samples), followed by all inputs for all sequences at time 2 (only for those that actually exist), etc.  Then, in addition, there needs to be a 

    int time_indexes[MAX_TIMESTEPS]
    
variable that points to the starting index of the batch for each timestep.

We won't include the code to do this here, as it involves a bit more bookkeeping to keep everything in place, but it should be apparent that for the cost of a bit more additional indexing, you can run the LSTM on _only_ those portions of a sequence that actually exist.  Whether or not this is ultimately beneficial depends on how much you're able to saturate the compute hardware at the later "sparser" stages of the processing.

## Training LSTMs: Truncated BPTT and hidden repackaging

Now that we have covered the basics of LSTM creation, we're going to briefly mention how to train these systems in practice.  As this involves actually running the LSTM code in a autodiff tool, we're going to instead just include pseudo-code for these ideas here, but you'll implement them within needle for the final homework.

First, we should emphasize one main point, that the majority of implementing training in a recurrent neural network "just" involves running the RNN under an automatic differentiation tool, and using autodiff to find the gradient of all parameters with respect to some loss.  That is, using roughly the notation from the above (i.e., with the `lstm` call that run over an entire (batch of) sequences, we could summarize the training procedure as the following:

```python
def train_lstm(X, Y, h0, c0, parameters)
    H, cn = lstm(X, h0, c0, parameters)
    l = loss(H, Y)
    l.backward()
    opt.step()
```
    
For a multi-layer LSTM, we actually have some choice in determinining in what "order" we run it: do we run a full layer first (over all time), and then iterate over depth?  Or do we run all depth first for a single time step, and then iterate over time?  While both are options, it's conceptually simpler (because we can re-use the same function above) to follow the first option, i.e., to just run each LSTM over the full sequence and then iterate over layers, i.e.,

```python
def train_lstm(X, Y, h0, c0, parameters)
    H = X
    for i in range(depth):
        H, cn = lstm(H, h0[i], c0[i], parameters[i])
    l = loss(H, Y)
    l.backward()
    opt.step()
```

This training process (for both single and multi-layer cases) is known as "backpropagation through time" (BPTT), as we're essentially doing backprop (but automatically via the autodiff framework) over each time step in the sequence.

### Long sequences and truncated BPTT

The process above works fine conceptually, but what happens in the case that we have a very long sequence?  For instance, in many language modeling tasks the true underlying sequence could be a document with thousands of words or audio signals that span many minutes.  Trying to train all of these in a "single" pass of BPTT would be:
    
1. Computationally/memory-wise infeasible (because we have to keep the whole computation graph in memory over the entire sequence).
2. Inefficient for learning.  Just like batch gradient descent is inefficient from a learning standpoint relative to SGD, taking a single gradient step for an entire sequence is very inefficient from a learning perspective: we have to do a ton of work to get a single parameter update.
    
Thus, the simple solution is just to divide the sequence in multiple shorter blocks.  That is, we train the the LSTM on segments of the full sequence.  This could look something like the following.

```python
for i in range(0,X.shape[0],BLOCK_SIZE):
    h0, c0 = zeros()
    train_lstm(X[i:i+BLOCK_SIZE], Y[i:i+BLOCK_SIZE], 
               h0, c0, parameters)
```

This works, and "solves" the problem of long sequence lengths. But it is also unsatisfying: we got rid of long sequences by just chopping them into shorter sequences.  And this ignores the fact that it is precisely the long term dependencies (beyond `BLOCK_SIZE`) that are often most interesting in sequence models, i.e., language models that "remember" the general context of the words they are generating, etc.

### Hidden repackaging

The way around this is to use what is called "hidden repackaging".  At the end of running an LSTM on a sequence, we have the final hidden units (hidden and cell units) of the LSTM.  These embed the "current" state of the system, in a way.  We can't continue to continue to differentiate through these variables in the autodiff graph, but we _can_ input their raw values as the $h_0$ and $c_0$ variables into the LSTM run on the next chunk of data.  To do this, we'd want to adjust our LSTM training code to return these variables, but detached from their gradients.

```python
def train_lstm(X, Y, h0, c0, parameters)
    H, cn = lstm(X, h0, c0, parameters)
    l = loss(H, Y)
    l.backward()
    opt.step()
    return H[-1].data, cn.data
```

We then use these values (instead of zeros), as the initial state of the LSTM in subsequent training loops.

```python
h0, c0 = zeros()
for i in range(0,X.shape[0],BLOCK_SIZE):
    h0, c0 = train_lstm(X[i:i+BLOCK_SIZE], Y[i:i+BLOCK_SIZE], h0, c0, parameters)
```

It's important to emphasize that this process is _still_ running truncated BPTT, as we're only computing gradients through a small portion of the full sequence.  But it's somewhat of a "middle ground" between doing full BPTT and always re-initializing the initial hidden states to zeros: the future LSTM states can get information from longer term context from the LSTM, and can use this to make its predictions of the future.  But it _cannot_ assess how changing the parameters of the LSTM would have changed this past initial state.